In [1]:
from pathlib import Path
import datetime as dt

import pandas as pd

from pybenchmark.client import BenchmarkClient

In [2]:
URL = "https://benchmark-container-app.braveflower-04bf6ce2.uksouth.azurecontainerapps.io/"
DATA_DIR = Path("../resources")

In [3]:
client = BenchmarkClient(url=URL)

# Muscle Groups

In [ ]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="musclegroups")
df.head()

In [ ]:
for name in df.name.tolist():
    try:
        response = client.musclegroups.create_musclegroup(name=name)
        print(f"Created Muscle Group {name}")
    except Exception as e:
        print(e)

# Exercises

In [ ]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="exercises")
df.head()

In [ ]:
mgs = pd.DataFrame(client.musclegroups.get_musclegroups())
mgs.head()

In [ ]:
for ix, exercise in df.iterrows():
    name = exercise["name"]
    mg_names = [mg.strip() for mg in exercise.musclegroups.split(",")]
    mg_ids = mgs[mgs["name"].isin(mg_names)].id.to_list()
    try:
        response = client.exercises.create_exercise(name=name, musclegroup_ids=mg_ids)
        print(f"Created Exercise {name}")
    except Exception as e:
        print(e)

# Workout Routines

In [ ]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workoutroutines")
df = df.rename(columns={"name": "routine_name"})
df.head()

In [ ]:
routines = df["routine_name"].unique()
routines

In [ ]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs.head()

In [ ]:
df["exercise_lower"] = df.exercise.str.lower()
df.head()

In [ ]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="name").set_index("routine_name")[["reps", "id"]]
df_all = df_all.reset_index().set_index(["routine_name", "id"])
df_all

In [ ]:
for routine in routines:
    all_exercises = []
    exercises = df_all.loc[routine].index.unique()
    for exercise in exercises:
        sets = df_all.loc[routine, exercise].to_dict(orient="records")
        data = {"id": exercise, "planned_sets": sets}
        all_exercises.append(data)
    try:
        response = client.workout_routines.create_workout_routine(name=routine, exercises=all_exercises)
        print(f"Created Routine {routine}")
    except Exception as e:
        print(e)

# Workouts

In [ ]:
df = pd.read_excel(DATA_DIR / "setup.xlsx", sheet_name="workouts")
df = df.rename(columns={"name": "routine_name"})
df.head()

In [ ]:
dates = df["date"].unique()
dates = [str(date.date()) for date in dates]

In [ ]:
exs = pd.DataFrame(client.exercises.get_exercises())
exs = exs.rename(columns={"id": "exercise_id", "name": "exercise_name"})
exs.head()

In [ ]:
routines = pd.DataFrame(client.workout_routines.get_workout_routines())
routines = routines.rename(columns={"name": "routine_name", "id": "routine_id"})
routines.head()

In [ ]:
df["exercise_lower"] = df.exercise.str.lower()
df["workoutroutine_lower"] = df.workoutroutine.str.lower()
df.head()

In [ ]:
df_all = df.merge(right=exs, left_on="exercise_lower", right_on="exercise_name")
df_all = df_all.merge(right=routines, left_on="workoutroutine_lower", right_on="routine_name")[["date", "weight", "reps", "routine_id", "exercise_id"]]
df_all = df_all.set_index(["date", "routine_id", "exercise_id"])
df_all

In [ ]:
for date in dates:
    all_workouts = []
    routines = df_all.loc[date].index.get_level_values(0).unique()
    for routine in routines:
        all_exercises = []
        exercises = df_all.loc[(date, routine)].index.unique()
        for exercise in exercises:
            sets = df_all.loc[(date, routine, exercise)].to_dict(orient="records")
            data = {"id": exercise, "sets": sets}
            all_exercises.append(data)
        try:
            response = client.workouts.create_workout(date=str(date), exercises=all_exercises, workoutroutine_id=routine)
            print(f"Created Workout for date {date}")
        except Exception as e:
            print(e)

# Users

In [25]:
user = {"name": "Jordan Begg", "email": "jordanbegg@email.com"}

In [26]:
try:
    response = client.users.create_user(**user)
    print(f"Created User {user['name']}")
except Exception as e:
    print(e)

Created User Jordan Begg


# Weights

In [4]:
WEEKDAYS = {
    "Monday": 0,
    "Tuesday": 1,
    "Wednesday": 2,
    "Thursday": 3,
    "Friday": 4,
    "Saturday": 5,
    "Sunday": 6
    }

In [5]:
df = pd.read_excel(DATA_DIR / "weights.xlsx", sheet_name="Sheet2")
df = df.melt(id_vars="Date", value_vars=["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"])
df["dow"] = df["variable"].map(WEEKDAYS)
df["Real Date"] = df.apply(axis=1, func = lambda x: dt.timedelta(days=x.dow) + x.Date)
df = df.drop(columns=["Date", "dow", "variable"]).rename(columns={"value": "weight", "Real Date": "date"}).dropna()
weights = df.to_dict(orient="records")
weights

[{'weight': 60.0, 'date': Timestamp('2022-02-07 00:00:00')},
 {'weight': 60.9, 'date': Timestamp('2022-02-14 00:00:00')},
 {'weight': 60.0, 'date': Timestamp('2022-02-28 00:00:00')},
 {'weight': 60.6, 'date': Timestamp('2022-03-14 00:00:00')},
 {'weight': 60.8, 'date': Timestamp('2022-03-21 00:00:00')},
 {'weight': 61.9, 'date': Timestamp('2022-03-28 00:00:00')},
 {'weight': 62.1, 'date': Timestamp('2022-04-25 00:00:00')},
 {'weight': 62.2, 'date': Timestamp('2022-05-30 00:00:00')},
 {'weight': 63.5, 'date': Timestamp('2022-06-13 00:00:00')},
 {'weight': 64.6, 'date': Timestamp('2022-06-20 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-07-04 00:00:00')},
 {'weight': 64.2, 'date': Timestamp('2022-07-11 00:00:00')},
 {'weight': 65.0, 'date': Timestamp('2022-08-22 00:00:00')},
 {'weight': 64.0, 'date': Timestamp('2022-08-29 00:00:00')},
 {'weight': 63.1, 'date': Timestamp('2022-09-05 00:00:00')},
 {'weight': 61.8, 'date': Timestamp('2022-09-12 00:00:00')},
 {'weight': 61.6, 'date'

In [18]:
for weight in weights:
    value = weight["weight"]
    date = weight["date"].date()
    user_id = 2
    try:
        response = client.weights.create_weight(weight=value, date=str(date), user_id=user_id)
        print(f"Created Weight {value}")
    except Exception as e:
        print(e)

Created Weight 60.0
Created Weight 60.9
Created Weight 60.0
Created Weight 60.6
Created Weight 60.8
Created Weight 61.9
Created Weight 62.1
Created Weight 62.2
Created Weight 63.5
Created Weight 64.6
Created Weight 64.0
Created Weight 64.2
Created Weight 65.0
Created Weight 64.0
Created Weight 63.1
Created Weight 61.8
Created Weight 61.6
Created Weight 60.7
Created Weight 58.7
Created Weight 58.8
Created Weight 57.9
Created Weight 56.7
Created Weight 58.2
Created Weight 59.8
Created Weight 61.8
Created Weight 61.0
Created Weight 61.4
Created Weight 61.4
Created Weight 61.4
Created Weight 61.3
Created Weight 62.0
Created Weight 62.4
Created Weight 61.7
Created Weight 62.6
Created Weight 61.9
Created Weight 62.5
Created Weight 63.2
Created Weight 63.2
Created Weight 62.1
Created Weight 63.6
Created Weight 63.3
Created Weight 65.8
Created Weight 66.8
Created Weight 67.1
Created Weight 67.7
Created Weight 67.9
Created Weight 69.5
Created Weight 69.7
Created Weight 70.5
Created Weight 71.6
